In [103]:
# accounts
import json
import solana
import time
import sys
import glob
from solana.rpc.api import Client
from solana.blockhash import Blockhash
from solana.publickey import PublicKey
from solana.system_program import CreateAccountWithSeedParams, create_account_with_seed, CreateAccountParams, create_account
from solana.transaction import Transaction, AccountMeta, TransactionInstruction
from solana.system_program import SYS_PROGRAM_ID
from spl.token.client import Token
from solana import keypair
from solana.publickey import PublicKey
from solana.sysvar import SYSVAR_RENT_PUBKEY

In [104]:
TOKEN_PROGRAM_ID: PublicKey = PublicKey("TokenkegQfeZyiNwAJbNbGKPFXCWuBvf9Ss623VQ5DA")

first_key_in_key_folder = glob.glob("../keys/*")[0]
payer_loaded_account = solana.keypair.Keypair(json.load(open(first_key_in_key_folder))[:32])


http_client = Client("https://api.devnet.solana.com")
payer_public_key = payer_loaded_account.public_key



# todo: change program_id to system program, change data size =0, change lampord for both alice and bob
tx = Transaction()
alice_keypair = solana.keypair.Keypair.generate()
alice_pubkey = alice_keypair.public_key



tx = Transaction()
bob_keypair = solana.keypair.Keypair.generate()
bob_pubkey = bob_keypair.public_key


print(f"Payer Account: {payer_public_key}")
print(f"alice Account: {alice_pubkey}")
print(f"bob Account: {bob_pubkey}")

Payer Account: 763cSApzBT5scjhSd6BdnkKkMYBhiQmPn2gctswkdkJM
alice Account: FLKPhbU8Kp8UdAuT8ZR2tfHM4dsCdSjhjabMZQ26JBX6
bob Account: JDregqoW8YpQK26qi7UomtBsdDJav7VQh2Rvo35yEdRG


In [105]:
X_mint_account_address = Token.create_mint(conn= http_client, payer = payer_loaded_account, mint_authority = payer_loaded_account.public_key, decimals = 0, program_id = TOKEN_PROGRAM_ID, )
Y_mint_account_address = Token.create_mint(conn= http_client, payer = payer_loaded_account, mint_authority = payer_loaded_account.public_key, decimals = 0, program_id = TOKEN_PROGRAM_ID, )
alice_X_token_account = X_mint_account_address.create_associated_token_account(alice_pubkey)
alice_Y_token_account = Y_mint_account_address.create_associated_token_account(alice_pubkey)
bob_X_token_account = X_mint_account_address.create_associated_token_account(bob_pubkey)
bob_Y_token_account = Y_mint_account_address.create_associated_token_account(bob_pubkey)

In [106]:
X_mint_account_address.pubkey

BJpUSqY51tbcqaAePVTomcLKTwoet5CsWUDcQ99E6Epq

In [107]:
Y_mint_account_address.pubkey

Hd1xF2Bp1BoEeF2q3d2j1Mwuiw4VACVcCYC7sjyoVXzG

In [108]:
bob_Y_token_account

yu3wkQ2fxFKirWaumqQGKYh96CeCqNhUNe5zWZZK2Kp

In [109]:
X_mint_account_address.mint_to(alice_X_token_account,payer_loaded_account,1000)
Y_mint_account_address.mint_to(bob_Y_token_account,payer_loaded_account,100)

{'jsonrpc': '2.0',
 'result': '62YgfoG1FrBee95CvnJHTURvuPFStbEtwo2XAMXuFYRas6aU8cgHGZ8sHSw8HBRwbh7keoxVoPzZXfLA2M1Mpr18',
 'id': 42}

In [132]:
import time
# http_client.request_airdrop(payer_loaded_account.public_key, 1000000000)
# time.sleep(20)
# http_client.request_airdrop(alice_pubkey, 1000000000)
# time.sleep(20)
# http_client.request_airdrop(bob_pubkey, 1000000000)
# print(http_client.get_balance(alice_pubkey))
# print(http_client.get_balance(bob_pubkey))
print(http_client.get_balance(payer_loaded_account.public_key))
# print(http_client.get_minimum_balance_for_rent_exemption(7)["result"])

{'jsonrpc': '2.0', 'result': {'context': {'slot': 86345480}, 'value': 20636858754}, 'id': 66}


In [111]:
# escrow meta-data account
deployed_program_key = glob.glob("../target/deploy/*.json")[0]
deployed_program_key_account = solana.keypair.Keypair(json.load(open(deployed_program_key))[:32])
program_id = deployed_program_key_account.public_key


tx = Transaction()
escrow_keypair = solana.keypair.Keypair.generate()
escrow_pubkey = escrow_keypair.public_key
data_size = 100
lamports = http_client.get_minimum_balance_for_rent_exemption(data_size)["result"]
transaction_instruction = create_account(
        params=CreateAccountParams(
            from_pubkey=payer_public_key, new_account_pubkey=escrow_pubkey,
            lamports=lamports, space=data_size, program_id=program_id
        )
    )

tx = tx.add(transaction_instruction)
transaction_results = http_client.send_transaction(tx, *[payer_loaded_account,escrow_keypair])
print(f"Program ID: {program_id}")

Program ID: 6QfnfdcHRHz3iucPW4nqeWkRR6ZfCACvajvxuqcduSFb


In [112]:
# todo, creat seed more specific
x_seeds = [
    b"vault_x",
    bytes(alice_pubkey),
    bytes(bob_pubkey),
    bytes(X_mint_account_address.pubkey),
    bytes(Y_mint_account_address.pubkey),
]

y_seeds = [
    b"vault_y",
    bytes(alice_pubkey),
    bytes(bob_pubkey),
    bytes(X_mint_account_address.pubkey),
    bytes(Y_mint_account_address.pubkey),
]

escrow_seeds = [
    b"escrow",
    bytes(alice_pubkey),
    bytes(bob_pubkey),
    bytes(X_mint_account_address.pubkey),
    bytes(Y_mint_account_address.pubkey),
]


vaultx, xseed = PublicKey.find_program_address(seeds=x_seeds,program_id=program_id)
vaulty, yseed = PublicKey.find_program_address(seeds=y_seeds,program_id=program_id)
escrow_address, escrow_seed = PublicKey.find_program_address(seeds=escrow_seeds,program_id=program_id)

In [113]:
print(vaultx)
print(vaulty)
print(escrow_address)

Eq4JRkqoK3uE7rM38PqKYo3FxfDJCwAZhC8w8wPoZ9zQ
icudHyzUS6tMq7jiskti8GERLeK1E9uphvZDuu1w9ey
H2zxnC3nyp9VaCn67BJDAqSnopMUBHALqeTXvz5NbEoz


In [114]:
# initialize transaction
from struct import *
data = pack('<BQQ', 0,13,15)

import struct
tx = Transaction()
tx_instruction = TransactionInstruction(
    keys=[AccountMeta(pubkey=escrow_address, is_signer=False, is_writable=True),
          AccountMeta(pubkey=X_mint_account_address.pubkey, is_signer=False, is_writable=False),
          AccountMeta(pubkey=Y_mint_account_address.pubkey, is_signer=False, is_writable=False),
          AccountMeta(pubkey=vaultx, is_signer=False, is_writable=True),
          AccountMeta(pubkey=vaulty, is_signer=False, is_writable=True),
          AccountMeta(pubkey=payer_public_key, is_signer=True, is_writable=False),
          AccountMeta(pubkey=alice_pubkey, is_signer=False, is_writable=False),
          AccountMeta(pubkey=bob_pubkey, is_signer=False, is_writable=False),
          AccountMeta(pubkey=TOKEN_PROGRAM_ID, is_signer=False, is_writable=False),
          AccountMeta(pubkey=SYSVAR_RENT_PUBKEY, is_signer=False, is_writable=False),
          AccountMeta(pubkey=SYS_PROGRAM_ID, is_signer=False, is_writable=False),
#           AccountMeta(pubkey=program_id, is_signer=False, is_writable=False),
         ],
    program_id=program_id,
    data=  data, #struct.pack("<LHB",1,1,1) # b58encode(bytes([9] * 7))#b'0001000100010001000100010001',
)

tx = tx.add(tx_instruction)


transaction_results = http_client.send_transaction(tx, *[payer_loaded_account])#, *[payer_loaded_account])

In [115]:
escrow_address

H2zxnC3nyp9VaCn67BJDAqSnopMUBHALqeTXvz5NbEoz

In [117]:
a = http_client.get_account_info(escrow_address)['result']['value']['data'][0]
import base64
unpack('<QQB', base64.b64decode(a)[:17])

(13, 15, 1)

In [118]:
escrow_address

H2zxnC3nyp9VaCn67BJDAqSnopMUBHALqeTXvz5NbEoz

In [119]:
# deposite
from struct import *
data = pack('<B', 1)

import struct
tx = Transaction()
tx_instruction = TransactionInstruction(
    keys=[AccountMeta(pubkey=escrow_address, is_signer=False, is_writable=True),
          AccountMeta(pubkey=X_mint_account_address.pubkey, is_signer=False, is_writable=False),
          AccountMeta(pubkey=Y_mint_account_address.pubkey, is_signer=False, is_writable=False),
          AccountMeta(pubkey=alice_X_token_account, is_signer=False, is_writable=True), # x_a info
          AccountMeta(pubkey=vaultx, is_signer=False, is_writable=True),
          AccountMeta(pubkey=alice_pubkey, is_signer=True, is_writable=False),
          AccountMeta(pubkey=bob_pubkey, is_signer=False, is_writable=False),
          AccountMeta(pubkey=TOKEN_PROGRAM_ID, is_signer=False, is_writable=False),
         ],
    program_id=program_id,
    data=  data,
)

tx = tx.add(tx_instruction)


transaction_results = http_client.send_transaction(tx, *[alice_keypair])#, *[payer_loaded_account])

In [120]:
alice_Y_token_account

FkbNsTyypkDa4KBhRcnMiwbuxTszkKyLVaJbSChsgGee

In [121]:
bob_X_token_account

BFmw2pWS4ZvScz6mnZsdNw85JUzsgXq9S1ACLe4GyJsx

In [122]:
alice_X_token_account

2pCbe1Mb6WmRYoMVzsUaYpoqdzQrerxHH618mSK2esjd

In [123]:
alice_X_token_account

2pCbe1Mb6WmRYoMVzsUaYpoqdzQrerxHH618mSK2esjd

In [135]:
# withdraw
from struct import *
data = pack('<B', 2)

import struct
tx = Transaction()
tx_instruction = TransactionInstruction(
    keys=[AccountMeta(pubkey=escrow_address, is_signer=False, is_writable=True),
          AccountMeta(pubkey=X_mint_account_address.pubkey, is_signer=False, is_writable=False),
          AccountMeta(pubkey=Y_mint_account_address.pubkey, is_signer=False, is_writable=False),
          AccountMeta(pubkey=alice_Y_token_account, is_signer=False, is_writable=True), # x_a info
          AccountMeta(pubkey=bob_X_token_account, is_signer=False, is_writable=True), # x_a info
          AccountMeta(pubkey=bob_Y_token_account, is_signer=False, is_writable=True), # x_a info
          AccountMeta(pubkey=vaultx, is_signer=False, is_writable=True),
          AccountMeta(pubkey=vaulty, is_signer=False, is_writable=True),
          AccountMeta(pubkey=alice_pubkey, is_signer=False, is_writable=False),
          AccountMeta(pubkey=bob_pubkey, is_signer=True, is_writable=False),
          AccountMeta(pubkey=TOKEN_PROGRAM_ID, is_signer=False, is_writable=False),
         ],
    program_id=program_id,
    data=  data,
)

tx = tx.add(tx_instruction)


transaction_results = http_client.send_transaction(tx, *[bob_keypair])#, *[payer_loaded_account])